# SAYFADAN VERİ ÇEKME

In [1]:
# Kullanılacak kütüphaneyi import etme
from selenium import webdriver

# Tarayıcıyı başlatma
browser = webdriver.Chrome()

# Hesap ayarlarda seçili özelliklerin kullanılabilmesi(filmlerin adının orjinal dilinde olması) için,
#sayfada hesap girişi yapılacak bilgiler
username = "***@gmail.com"
password = "****"

# Yüklenecek web sayfası
browser.get("https://www.imdb.com/")

# Hesap girişi yapılabilmesi için olan işlemler
browser.find_element("xpath", f'//*[@id="imdbHeader"]/div[2]/div[5]/a/span').click()
browser.find_element("xpath", f'//*[@id="signin-options"]/div/div[1]/a[1]').click()
username_input = browser.find_element("xpath", f'//*[@id="ap_email"]')
password_input = browser.find_element("xpath", f'//*[@id="ap_password"]')

username_input.send_keys("****.com")
password_input.send_keys("****")

browser.find_element("xpath", f'//*[@id="signInSubmit"]').click()

browser.refresh()

# IMDB sitesinde önerilen ilk 1000 film adresine gider
browser.get("https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating") 

# Filmleri içerisinde tutacak dizin tanımlama
film_list = [] 

# Sayfa geçişleri için döngü elemanları
i = 1
j = 1
k = 1

# Her sayfada 100 tane film olan 10 sayfanın sayfalar arası geçiş kodu
while i <= 5:
    if (i % 100 == 1 and i != 1): 
        if j == 1:
            browser.find_element("xpath", f'//*[@id="main"]/div/div[4]/a').click()
            i = 1
            j += 1
            k += 1
        else:
            if k == 10:
                break
            else:
                browser.find_element("xpath", f'//*[@id="main"]/div/div[4]/a[2]').click()
                i = 1
                k += 1
    
    browser.implicitly_wait(0.5)
    
    # İstenilen film bilgilerini sayfada bulma
    try:

        titles = browser.find_elements("xpath", f'/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[3]/div/div[{i}]/div[3]/h3/a')
        imdbs = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/div/div[1]/strong')
        typess = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[5]')
        years = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/h3/span[2]')
        stars_dir = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[3]')
        durations = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[3]')
        certificates = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[1]/span[1]')
        descriptions = browser.find_elements("xpath", f'//*[@id="main"]/div/div[3]/div/div[{i}]/div[3]/p[2]')


        # İstenilen film bilgilerini zip dosyasına dönüştürme
        for title,imdb,typei,year,star_dir,duration,certificate,description,in zip(titles, imdbs, typess, years,
                                                                                        stars_dir,durations,certificates,descriptions):
            
            film_titles = title.text
            film_imdbs = imdb.text
            film_types = typei.text
            film_years = year.text
            film_stars_dir = star_dir.text
            film_durations = duration.text
            film_certificates = certificate.text
            film_descriptions = description.text
            film_list.append([film_titles,  film_imdbs, film_types, film_years, 
                              film_stars_dir,film_durations,film_certificates,film_descriptions])
            
        i += 1 
        
    except Exception as e:
        print(f"Hata oluştu: {e}")


# CSV DOSYASINA DÖNÜŞTÜRME

In [3]:
import csv

# Çıktıları istediğimiz kolon adları ile CSV dosyasına kaydetmek için
with open('film_list.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["title","imdb","type","year","dir_star","duration","certificate","description"])  # Kolon isimleri
    writer.writerows(film_list) # CSV dosyası adı